In [19]:
from datetime import datetime

import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px

def sr(ret):
    ret = ret[ret!=0]
    print(f'SR: {np.mean(ret) / np.std(ret) * np.sqrt(252)}')

def cagr(cpnl):
    t = ((cpnl.index[-1] - cpnl.index[0]) / 365.25)
    t = (t.seconds / (24 * 60 * 60) + t.days)
    FV = cpnl.values[-1]
    print(f'CAGR: {FV ** (1/t) - 1}')

In [ ]:
from datetime import datetime

import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px

def sr(ret):
    ret = ret[ret!=0]
    print(f'SR: {np.mean(ret) / np.std(ret) * np.sqrt(252)}')

def cagr(cpnl):
    t = ((cpnl.index[-1] - cpnl.index[0]) / 365.25)
    t = (t.seconds / (24 * 60 * 60) + t.days)
    FV = cpnl.values[-1]
    print(f'CAGR: {FV ** (1/t) - 1}')

https://www.quantifiedstrategies.com/calendar-effects-long-term-treasuries/#Rest_of_the_month_effect_in_long-term_Treasuries

In [2]:
datetime.now()

datetime.datetime(2023, 4, 7, 9, 40, 26, 378012)

In [50]:
raw_prices_df = yf.download(tickers='TLT',
                        end=datetime.now())


[*********************100%***********************]  1 of 1 completed


In [51]:
raw_prices_df['sma200'] = raw_prices_df['Adj Close'].rolling(200).mean()


In [52]:
px.line(raw_prices_df.melt(ignore_index=False, value_vars=['Adj Close', 'sma200']), y='value', color='variable')

In [53]:
#### buy at close of the 7th trading day in a month
#### sell at close of the month


In [54]:
prices_df = raw_prices_df.copy()
prices_df['ret'] = prices_df['Adj Close'].pct_change().shift(-1)
prices_df['vol'] = prices_df.ret.rolling(22).std()
prices_df['month'] = prices_df.index.month
prices_df['year'] = prices_df.index.year
# prices_df['buy'] = 1
# prices_df['sell'] = 0
buy = prices_df.reset_index().groupby(['year', 'month']).nth(7)
buy['buy'] = 1
prices_df = pd.merge(left = prices_df, left_index = True, right=buy[['Date', 'buy']].set_index('Date'), right_index=True, how = 'left')
buy = prices_df.reset_index().groupby(['year', 'month']).nth(-1)
buy['sell'] = -1
prices_df = pd.merge(left = prices_df, left_index = True, right=buy[['Date', 'sell']].set_index('Date'), right_index=True, how = 'left')
prices_df.buy.fillna(0, inplace=True)
prices_df.sell.fillna(0, inplace=True)
prices_df['trade'] = prices_df.buy + prices_df.sell

# prices_df.set_index('Date', inplace=True)


In [55]:
st_trade = prices_df.loc[prices_df.trade != 0].iloc[0]
if st_trade.trade == -1:
    print('1st trade is a close')
    mask = prices_df.index > st_trade.name
    prices_df = prices_df.loc[mask]

prices_df['position'] = 1 #prices_df.trade.cumsum()

1st trade is a close


In [56]:
prices_df['pnl'] = prices_df.ret * prices_df.position
prices_df['cpnl'] = (prices_df.pnl + 1).cumprod()
sr(prices_df.pnl.dropna().values)
cagr(prices_df.cpnl.dropna())

SR: 0.3958452662122207
CAGR: 0.0473868327252609


In [57]:
px.line(prices_df, y='cpnl', log_y = True)

In [33]:
prices_df.loc[prices_df['Adj Close'] < prices_df['sma200'], 'pnl'] = 0
prices_df['cpnl'] = (prices_df.pnl + 1).cumprod().ffill()
sr(prices_df.pnl.dropna().values)
cagr(prices_df.cpnl)
px.line(prices_df, y='cpnl', log_y = True)

SR: 0.388907873688347
CAGR: 0.02907723703579279


In [34]:
prices_df[['vol', 'ret']].corr()

,vol,ret
vol,1.000000,0.011667
ret,0.011667,1.000000


In [35]:
raw_prices_df = yf.download(tickers=['TLT', 'SPY'],
                        end=datetime.now())

[*********************100%***********************]  2 of 2 completed


In [36]:
ret_df = raw_prices_df['Adj Close'].pct_change()
ret_df.corr()

,SPY,TLT
SPY,1.000000,-0.355264
TLT,-0.355264,1.000000


In [37]:
corr_df = ret_df.rolling(window = 66).corr().iloc[:,1].dropna()
corr_df = corr_df.reset_index(level=1, drop=True)
px.line(corr_df.loc[corr_df.values < 0.99])

In [45]:
corr_df[corr_df.values < 0.99]

Date
2002-10-31   -0.644104
2002-11-01   -0.660574
2002-11-04   -0.659604
2002-11-05   -0.652941
2002-11-06   -0.649410
                ...   
2023-03-31    0.010394
2023-04-03   -0.000889
2023-04-04   -0.015797
2023-04-05   -0.041261
2023-04-06   -0.046992
Name: TLT, Length: 5143, dtype: float64

In [61]:
prices_df['corr'] = corr_df[corr_df.values < 0.99].shift()

In [62]:
px.line(prices_df.melt(ignore_index=False, value_vars=['corr', 'cpnl']), y='value', color='variable')

In [63]:

px.scatter(prices_df, x='corr', y='ret')

In [76]:
prices_df.loc[prices_df['corr'] < 0.05, ['corr', 'ret']].corr()

,corr,ret
corr,1.000000,-0.036319
ret,-0.036319,1.000000
